# True parameters sets for OpenMM Simulations

### Defining reproducible and serializable simulation parameters
These provide a means to exchange and cache information about how you set up a simulation \
These can be readily serialized to disc and are stored at the start of each simulation, if run through polymerist

In [ ]:
from openmm.unit import femtosecond, picosecond, nanosecond
from openmm.unit import kelvin, atmosphere
from polymerist.mdtools.openmmtools.parameters import (
    SimulationParameters,
    ThermoParameters,
    IntegratorParameters,
    ReporterParameters,
)
from polymerist.mdtools.openmmtools.reporters import DEFAULT_STATE_DATA_PROPS


# define how data should be periodically reported during a simulation (will be shared between simulations in this examples)
reporter_params = ReporterParameters( # these will be shared between both sets of parameters
        report_trajectory=True,
        traj_ext='dcd', # output to compressed binary trajectory files (recommended)
        report_state_data=True,
        state_data=DEFAULT_STATE_DATA_PROPS, # can tune these to taste
        report_checkpoint=True, # also keep checkpoints of OpenMM objects (specific to Context and machine)
        report_state=True,      # saving State is a bit redundant with checkpoints, but is machine-transferrable
    )

# define reproducible simulation parameter sets
## 1) HIGH-TEMPERATURE ANNEAL FOR RELAXATION
equil_params = SimulationParameters( 
    integ_params=IntegratorParameters(
        time_step=1*femtosecond,
        total_time=100*picosecond, # just a short simulation to demonstrate
        num_samples=10, # don't want to take too many samples
    ),
    thermo_params=ThermoParameters(
        ensemble='NPT', #
        temperature=600*kelvin,
        friction_coeff=1*picosecond**-1, # required for Langevin Thermostat
        barostat_freq=25, # number of steps between barostat move attempts
    ),
    reporter_params=reporter_params,
)

# TODO: equilibration?

## 2) LOW-TEMPERATURE PRODUCTION
prod_params = SimulationParameters( 
    integ_params=IntegratorParameters(
        time_step=2*femtosecond,  # NOTE that 2 fs step doesn't work well for unconstrainted FFs
        total_time=0.5*nanosecond, # just a short simulation to demonstrate
        num_samples=50, # don't want to take too many samples
    ),
    thermo_params=ThermoParameters(
        ensemble='NVT', # clamp volume
        temperature=300*kelvin,
        friction_coeff=1*picosecond**-1, 
    ),
    reporter_params=reporter_params,
)